<a href="https://colab.research.google.com/github/eunzzae/Study_NLP/blob/main/231202_%EC%8B%A4%EC%8A%B5_longseq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 다음 문자열을 예측하는 RNN

In [1]:
import torch
import torch.optim as optim
import numpy as np

In [2]:
# Random seed to make results deterministic and reproducible
torch.manual_seed(0)

## 학습 문장 생성

In [3]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [4]:
sentence

"if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea."

* char(알파벳) 단위로 문자들을 생성한다.
* 똑같이 vacab을 만들자.

In [5]:
# make dictionary
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}

In [6]:
# hyperparameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
sequence_length = 10
learning_rate = 0.01

* 텍스트 생성을 위해서 아래와 같은 데이터 구성을 제작한다.
* 입력된 시퀀스들이 다음 시퀀스들로 연결한다.
* 문장을 일정 길이로 잘라서 한칸씩 미루는 구현을 한다.

```
0 if you wan -> f you want
1 f you want ->  you want
```

In [7]:
# data setting
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
   x_str = sentence[i:i+sequence_length]
   y_str = sentence[i+1:i+sequence_length+1]
   print(i, x_str, '->', y_str)

   x_data.append([char_dic[c] for c in x_str])
   y_data.append([char_dic[c] for c in y_str])

x_one_hot = [np.eye(dic_size)[x] for x in x_data] # 주어진 크기의 단위행렬을 생성하는 np 함수이며, 단위행렬은 주 대각선이 모두 1이고, 나머지 원소는 0인 정사각 행렬이다.

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [8]:
x_data[0]

[4, 5, 13, 1, 3, 17, 13, 8, 16, 20]

In [9]:
x_one_hot[0]

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

## Data Embedding
* one hot 형태로 전처리 된 데이터를 임베딩한다.
* torch에서 학습하기 위해서 torch tensor로 변경한다.

In [10]:
# transform as torch tensro variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

<ipython-input-10-3df5115ff887>:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  X = torch.FloatTensor(x_one_hot)


In [11]:
# declare RNN + FC
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(dic_size, hidden_dim, bias=True) ## ??? hidden dim이 두 번 들어간 이
    def forward(self, x):
        x, _hidden = self.rnn(x)
        x = self.fc(x)
        return(x)

In [12]:
net = Net(dic_size, hidden_size, 2)

## 모델 학습 파트
* 단어를 예측하는 파트는 '다중분류'이다.
* 모델의 마지막 Activation이 없는 상태로 Loss 값으로 전달된다.

In [13]:
# loss&optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

## 테스트를 위해서 위에 구성한 169개의 데이터가 하나의 배치로 들어간다.

In [14]:
# start training
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2) # 최대값을 가지는 원소의 인덱스를 반환한다.
    predict_str = "i" # 최초 문자열을 준 후 학습을 시장하면 더 정확한 예측이 가능하다.
    for j, result in enumerate(results):
        if j == 0:
            predict_str += ''.join([char_set[t] for t in result]) # result리스트에 있는 각 예측 토큰을 'char_set'을 사용하여 해당 문자열로 변환하고, 이를 predict_str에 추가한다.
        else:
            predict_str += char_set[result[-1]]
    print(predict_str)

io   .p .p.o k. c.    p .pk  ....pp  p..  .  ..p.p k.pgkpp. .. ..ppppk  ...pp.   ..pp ppp  . k.p. k.p p p.   . .. .p.  .o.p k..p..k..p. k. ..pp k.p ..p k. p pp k p .p    .p k.p p p
i                                                                                                                                                                                   
i                                                                                                                                                                                   
i                                                                                                                                                                                   
i                                                                                                                                                                                   
i                                                                                              